In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

test = pd.read_excel("C:/Users/joser/Downloads/test.csv.xlsx")
train = pd.read_excel("C:/Users/joser/Downloads/train.csv.xlsx")



In [2]:
train["Familiares"] = train["SibSp"] + train["Parch"]
test["Familiares"] = test["SibSp"] + test["Parch"]

train["Familiares"].describe() #Esta variable muestra con cuantos familiares viajaba cada pasajero

# Agregar columna 'source' para identificar el origen
train["source"] = "train"
test["source"] = "test"

# Agregar columna Survived a test con valores vacíos
test["Survived"] = np.nan

# Add 'Familiares' column to test before concatenating
test["Familiares"] = test["SibSp"] + test["Parch"]

# Unir verticalmente
combined = pd.concat([train, test], axis=0, ignore_index=True)

print("Tamaño del conjunto combinado:", combined.shape)
display(combined.head())

Tamaño del conjunto combinado: (1309, 14)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Familiares,source
0,1,0.0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.25,NaN,S,1,train
1,2,1.0,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,712833.00,C85,C,1,train
2,3,1.0,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7925.00,NaN,S,0,train
3,4,1.0,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.10,C123,S,1,train
4,5,0.0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.05,NaN,S,0,train


In [4]:
data = combined.copy()
# Verificamos el tamaño de cada conjunto
print("Tamaño de los conjuntos:")
print(data["source"].value_counts())


# Estadísticas generales por conjunto (train y test)
print("Estadísticas descriptivas por conjunto (numéricas):")
print(data.groupby("source").describe())



# Comparamos la proporción de hombres y mujeres en cada conjunto
print("Distribución de sexo por conjunto:")
print(data.groupby("source")["Sex"].value_counts(normalize=True))


# Comparamos clases de boleto (Pclass) en cada conjunto
print("Distribución de clases por conjunto:")
print(data.groupby("source")["Pclass"].value_counts(normalize=True))


# Comparamos edad promedio
print("Edad promedio por conjunto:")
print(data.groupby("source")["Age"].mean())



# Comparamos tarifas promedio
print("Tarifa promedio (Fare) por conjunto:")
print(data.groupby("source")["Fare"].mean())

Tamaño de los conjuntos:
source
train    891
test     418
Name: count, dtype: int64
Estadísticas descriptivas por conjunto (numéricas):
       PassengerId                                                      \
             count    mean         std    min     25%     50%      75%   
source                                                                   
test         418.0  1100.5  120.810458  892.0  996.25  1100.5  1204.75   
train        891.0   446.0  257.353842    1.0  223.50   446.0   668.50   

               Survived            ...     Fare            Familiares  \
           max    count      mean  ...      75%        max      count   
source                             ...                                  
test    1309.0      0.0       NaN  ...  78958.0  5123292.0      418.0   
train    891.0    891.0  0.383838  ...  78958.0  5123292.0      891.0   

                                                      
            mean       std  min  25%  50%  75%   max  
source           

In [4]:
#Se muestran las columnas con valores faltantes (NaN), como Age, Cabin, Embarked o Fare.
faltantes = combined.isnull().sum()
print("Valores faltantes por columna:")
print(faltantes[faltantes > 0])

Valores faltantes por columna:
Survived     418
Age          263
Fare           1
Cabin       1014
Embarked       2
dtype: int64


In [ ]:
#  Edad promedio
print("Edad promedio:", train["Age"].mean())

#  Sobrevivientes y fallecidos
print(train["Survived"].value_counts())

#  Tarifa promedio en primera clase
print("Tarifa promedio 1ra clase:", train.loc[train["Pclass"] == 1, "Fare"].mean())

#  Edad más joven y más vieja
print("Edad mínima:", train["Age"].min(), " | Edad máxima:", train["Age"].max())

#  Pasajeros por puerto
print(train["Embarked"].value_counts())

Edad promedio: 29.69911764705882
Survived
0    549
1    342
Name: count, dtype: int64
Tarifa promedio 1ra clase: 383077.2
Edad mínima: 0.42  | Edad máxima: 80.0
Embarked
S    644
C    168
Q     77
Name: count, dtype: int64


In [6]:
combined["Familiares"] = combined["SibSp"] + combined["Parch"]

# Pasajeros con al menos 1 familiar
con_familia = combined[combined["Familiares"] > 0]
print("número de pasajeros con familiares a bordo:", con_familia.shape[0])

# Crear una nueva columna indicadora
combined["viajo_solo"] = combined["Familiares"] == 0

# Contar proporciones y totales
print("\nConteo de pasajeros que viajaron solos vs con familia:")
print(combined["viajo_solo"].value_counts())

print("\nProporciones:")
print(combined["viajo_solo"].value_counts(normalize=True))

número de pasajeros con familiares a bordo: 519

Conteo de pasajeros que viajaron solos vs con familia:
viajo_solo
True     790
False    519
Name: count, dtype: int64

Proporciones:
viajo_solo
True     0.603514
False    0.396486
Name: proportion, dtype: float64


In [7]:
# Crear la nueva variable "Familiares" sumando SibSp y Parch
combined["Familiares"] = combined["SibSp"] + combined["Parch"]

# Crear la columna "GrupoFamilia" según el número de familiares
grupo = []
for i in combined["Familiares"]:
    if i == 0:
        grupo.append("Grupo 1: sin familiares")
    elif i >= 1 and i <= 3:
        grupo.append("Grupo 2: familias pequeñas")
    else:
        grupo.append("Grupo 3: familias grandes")

combined["GrupoFamilia"] = grupo

# Crear columna que diga si la persona tenía cabina o no
combined["TieneCabina"] = combined["Cabin"].notna()

# Contar cuántas personas hay en cada grupo familiar
conteo = combined.groupby("GrupoFamilia")["TieneCabina"].value_counts()

print("Distribución de cabinas por grupo familiar:\n")
print(conteo)

# También ver cuántas personas totales hay en cada grupo
print("\nCantidad total por grupo familiar:\n")
print(combined["GrupoFamilia"].value_counts())


Distribución de cabinas por grupo familiar:

GrupoFamilia                TieneCabina
Grupo 1: sin familiares     False          659
                            True           131
Grupo 2: familias pequeñas  False          284
                            True           153
Grupo 3: familias grandes   False           71
                            True            11
Name: count, dtype: int64

Cantidad total por grupo familiar:

GrupoFamilia
Grupo 1: sin familiares       790
Grupo 2: familias pequeñas    437
Grupo 3: familias grandes      82
Name: count, dtype: int64


In [8]:
datos = combined.copy()
#Copiar el DataFrame combinado
grupo_edad = []
#Separacion de los grupos basandose en la edad
for e in datos["Age"]:
    if pd.isna(e):
        grupo_edad.append("N.R")
    elif e <= 10:
        grupo_edad.append("Niño")
    elif e >= 18 and e <= 50:
        grupo_edad.append("Adulto")
    elif e > 50:
        grupo_edad.append("Mayor")
    else: 
        grupo_edad.append("Adolecente")
#Asignar columna de "Grupo etario"
datos["grupo_etario"] = grupo_edad
#Crear columna para comprobar si tienen columna o no 
datos["Tiene_cabina"] = datos["Cabin"].notna()
#Ver cuántas personas sobrevivieron o no, por grupo etario, sexo y cabina
tabla = datos.groupby(["grupo_etario", "Sex", "TieneCabina"])["Survived"].value_counts()

print("\nDistribución de supervivencia por grupo etario, sexo y cabina:\n")
print(display(tabla))

tasa = datos.groupby(["grupo_etario", "Sex", "TieneCabina"])["Survived"].mean()

print("\nTasa (proporción) de supervivencia en cada grupo:\n")
print(display(tasa))


Distribución de supervivencia por grupo etario, sexo y cabina:



grupo_etario  Sex     TieneCabina  Survived
Adolecente    female  False        1.0          12
                                   0.0           5
                      True         1.0           7
              male    False        0.0          21
                                   1.0           2
                      True         1.0           2
Adulto        female  False        1.0          80
                                   0.0          43
                      True         1.0          63
                                   0.0           3
              male    False        0.0         245
                                   1.0          36
                      True         0.0          39
                                   1.0          28
Mayor         female  False        1.0           4
                      True         1.0          12
                                   0.0           1
              male    False        0.0          26
                                   1.0

None

Tasa (proporción) de supervivencia en cada grupo:



grupo_etario  Sex     TieneCabina
Adolecente    female  False          0.705882
                      True           1.000000
              male    False          0.086957
                      True           1.000000
Adulto        female  False          0.650407
                      True           0.954545
              male    False          0.128114
                      True           0.417910
Mayor         female  False          1.000000
                      True           0.923077
              male    False          0.037037
                      True           0.250000
N.R           female  False          0.630435
                      True           1.000000
              male    False          0.107143
                      True           0.333333
Niño          female  False          0.629630
                      True           0.500000
              male    False          0.481481
                      True           1.000000
Name: Survived, dtype: float64

None
